In [9]:
import pyopencl as cl
import numpy as np

# Новые начальные условия и параметры
t0 = 0.0  # начальное значение t
y0 = 2.0  # новое начальное значение y
h = 0.02  # новый шаг интегрирования
num_steps = 1000  # новое количество шагов

# Инициализация OpenCL контекста и очереди
context = cl.create_some_context()
queue = cl.CommandQueue(context)

# Инициализация массива для хранения результата
y = np.zeros(num_steps + 1).astype(np.float32)
y[0] = y0

# Создание буфера для передачи данных в OpenCL устройство
mf = cl.mem_flags
buffer_y = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=y)

# Код ядра для метода Рунге-Кутты
kernel_code = """
__kernel void RungeKutta(
    __global float* y,
    const int num_steps,
    const float t0,
    const float h)
{
    float k1, k2, k3, k4;
    float yi = y[0];
    float t = t0;

    for (int i = 0; i < num_steps; ++i)
    {
        k1 = h * (-yi);
        k2 = h * (-(yi + k1 / 2.0f));
        k3 = h * (-(yi + k2 / 2.0f));
        k4 = h * (-(yi + k3));

        yi = yi + (k1 + 2.0f * k2 + 2.0f * k3 + k4) / 6.0f;
        y[i+1] = yi;
        t = t + h;
    }
}
"""

# Сборка и запуск программы OpenCL
program = cl.Program(context, kernel_code).build()
RungeKutta_kernel = program.RungeKutta
RungeKutta_kernel.set_args(buffer_y, np.int32(num_steps), np.float32(t0), np.float32(h))

global_size = (num_steps,)
cl.enqueue_nd_range_kernel(queue, RungeKutta_kernel, global_size, None).wait()

# Получение результата с устройства
cl.enqueue_copy(queue, y, buffer_y).wait()

# Вывод результатов
print("Результат решения простого дифференциального уравнения методом Рунге-Кутты:")
for i in range(10):
    print(f"y({t0 + i * h}) = {y[i]}")


Результат решения простого дифференциального уравнения методом Рунге-Кутты:
y(0.0) = 2.0
y(0.02) = 1.9603973627090454
y(0.04) = 1.9215788841247559
y(0.06) = 1.8835290670394897
y(0.08) = 1.8462326526641846
y(0.1) = 1.8096747398376465
y(0.12) = 1.7738407850265503
y(0.14) = 1.7387163639068604
y(0.16) = 1.7042875289916992
y(0.18) = 1.6705403327941895
